In [43]:
pip install statsmodels


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [52]:
import scipy.stats
import statsmodels.api as sm
import statsmodels.formula.api as smf
import os
import pandas as pd
import numpy as np
import stats

In [63]:
# Set random seed.
np.random.seed(42)

# Read the processed CSV file.
df = pd.read_csv("ms_data.csv")

# Convert visit_date to datetime.
df['visit_date'] = pd.to_datetime(df['visit_date'])

#  Sort by patient_id and visit_date.
df = df.sort_values(by = ['patient_id', 'visit_date'])

# Read insurance types from `insurance.lst`.
with open('insurance.lst', 'r') as f:
    insurance_types = [line.strip() for line in f.readlines()]

# Randomly assign (but keep consistent per patient_id).
unique_patients = df['patient_id'].unique()
patient_insurance_map = {patient_id: np.random.choice(insurance_types) for patient_id in unique_patients}
df['insurance_type'] = df['patient_id'].map(patient_insurance_map)

# Generate visit costs based on insurance type. Different plans have different effects on cost.
base_costs = {'Medicare': 100,
    'Medicaid': 200,
    'Private': 50,
    'Other': 500
}

# Add random variation.
variation_factor = 0.2 # 20% variation
df['visit_cost'] = df['insurance_type'].map(base_costs) * (1 + np.random.uniform(-variation_factor, variation_factor, len(df))).round(3)

# Set appropriate data types.
df['patient_id'] = df['patient_id'].astype(str)
df['education_level'] = df['education_level'].astype(str)
df['insurance_type'] = df['insurance_type'].astype(str)

# Spring: March, April, May (months 3, 4, 5)
# Summer: June, July, August (months 6, 7, 8)
# Fall: September, October, November (months 9, 10, 11)
# Winter: December, January, February (months 12, 1, 2)

# Add a 'season' column based on the month of the visit_date
def get_season(month):
    if month in [3, 4, 5]:
        return 'Spring'
    elif month in [6, 7, 8]:
        return 'Summer'
    elif month in [9, 10, 11]:
        return 'Fall'
    else:
        return 'Winter'

# Apply the get_season function to the 'visit_date' to categorize the season
df['season'] = df['visit_date'].dt.month.apply(get_season)

df.head()

,patient_id,visit_date,age,education_level,walking_speed,insurance_type,visit_cost,season
0,P0001,2020-01-23,34.28,Bachelors,4.40,Private,53.95,Winter
1,P0001,2020-04-16,34.51,Bachelors,4.19,Private,50.70,Spring
2,P0001,2020-07-03,34.72,Bachelors,4.71,Private,46.20,Summer
3,P0001,2020-10-15,35.00,Bachelors,4.86,Private,56.30,Fall
4,P0001,2020-12-29,35.21,Bachelors,4.50,Private,53.70,Winter


In [64]:
df['visit_date'] = pd.to_datetime(df['visit_date'])
df['patient_id'] = df['patient_id'].astype(str)
df['education_level'] = df['education_level'].astype(str)
df['age'] = df['age'].astype(float)
df['walking_speed'] = df['walking_speed'].astype(float)
df['visit_cost'] = df['visit_cost'].astype(float)
df['season'] = df['season'].astype(str)
print(df.dtypes)

print(f"Number of missing values in each column: {df.isnull().sum()}")
print(f"Number of rows with at least one missing value: {df.isnull().any(axis=1).sum()}")

patient_id                 object
visit_date         datetime64[ns]
age                       float64
education_level            object
walking_speed             float64
insurance_type             object
visit_cost                float64
season                     object
dtype: object
Number of missing values in each column: patient_id         0
visit_date         0
age                0
education_level    0
walking_speed      0
insurance_type     0
visit_cost         0
season             0
dtype: int64
Number of rows with at least one missing value: 0


In [65]:
# 1. Analyze walking speed:
#    - Multiple regression with education and age (report coeffcients and confidence intervals)
#    - Account for repeated measures
#    - Test for significant trends

import statsmodels.api as sm

# Prepare the data as before
X = df[['age']]
X = pd.get_dummies(df['education_level'], drop_first=True).join(X)  # Convert education_level to dummy variables
X = sm.add_constant(X)  # Add intercept

y = df['walking_speed']

# Ensure that X and y are purely numeric arrays
X = X.astype(float) 
y = y.astype(float)

# Fit the OLS model
model = sm.OLS(y, X).fit()

# Clustered standard errors by subject (assuming 'patient_id' is the subject identifier)
clustered_se = model.get_robustcov_results(cov_type='cluster', groups=df['patient_id'])

# Display the summary with clustered standard errors
print(clustered_se.summary())

# Clustered Standard Errors: The cov_type='cluster' argument in get_robustcov_results accounts for the fact that there are repeated measurements for each subject. The groups=df['id'] argument specifies that the clustering should be done by the subject identifier (id).

# Interpretation: The coefficients remain the same, but now the standard errors are adjusted for the repeated measurements, which leads to more accurate statistical tests (e.g., p-values).

                            OLS Regression Results                            
Dep. Variable:          walking_speed   R-squared:                       0.807
Model:                            OLS   Adj. R-squared:                  0.807
Method:                 Least Squares   F-statistic:                 2.059e+04
Date:                Tue, 12 Nov 2024   Prob (F-statistic):               0.00
Time:                        21:17:52   Log-Likelihood:                -5411.7
No. Observations:               15431   AIC:                         1.083e+04
Df Residuals:                   15426   BIC:                         1.087e+04
Df Model:                           4                                         
Covariance Type:              cluster                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const            5.5992      0.008    690.553   

In [62]:
print(clustered_se.params) # Coefficients 
print(clustered_se.pvalues) # P-values
print(clustered_se.conf_int()) # Confidence Intervals
print(clustered_se.rsquared_adj)

[ 5.5991894   0.41524264 -0.792317   -0.39032524 -0.03013812]
[0.00000e+000 0.00000e+000 0.00000e+000 3.91879e-318 0.00000e+000]
[[ 5.5832782   5.61510059]
 [ 0.40164808  0.4288372 ]
 [-0.80557048 -0.77906352]
 [-0.40368752 -0.37696296]
 [-0.03040785 -0.02986839]]
0.8069489306941992


In [58]:
# Test for trends.

X = df[['age']]
X = pd.get_dummies(df['education_level'], drop_first=True).join(X)  # Convert education_level to dummy variables
X = sm.add_constant(X)  # Add intercept
y = df['walking_speed']

# Ensure that X and y are purely numeric arrays
X = X.astype(float) 
y = y.astype(float)

# How the effect of age changes based on education level.
X['age_education_interaction'] = X['age'] * X['education_level']
est = sm.OLS(y, X).fit()
print(est.summary())

KeyError: 'education_level'

In [ ]:
# 2. Analyze costs:
#    - Simple analysis of insurance type effect
#    - Box plots and basic statistics (report coeffcients and confidence intervals)
#    - Calculate effect sizes

In [ ]:
# 3. Advanced analysis:
#    - Education age interaction effects on walking speed
#    - Control for relevant confounders
#    - Report key statistics and p-values (report coeffcients and confidence intervals)